# **Homework 1: COVID-19 Cases Prediction (Regression)**

In [ ]:
# 装载google盘
from google.colab import drive
drive.mount('/content/gdrive')

目标：
* 使用深度神经网络 （DNN） 解决回归问题。
* 了解基本的 DNN 培训技巧。
* 熟悉PyTorch。

如果您有任何疑问，请通过TA时间，NTU COOL或电子邮件联系TA，mlta-2022-spring@googlegroups.com

# 下载数据
如果下面的 Google 云端硬盘链接不起作用，您可以从 [Kaggle](https://www.kaggle.com/t/a3ebd5b5542f0f55e828d4f00de8e59a) 下载数据，然后手动将数据上传到工作区。

In [ ]:
# 下载训练集和测试集
!gdown --id '1kLSW_-cW2Huj7bh84YTdimGBOJaODiOS' --output covid.train.csv
!gdown --id '1iiI5qROrAhZn-o4FPqsE97bMzDEFvIdg' --output covid.test.csv

# 导包

In [ ]:
# 数学操作
import math
import numpy as np

# 读写数据
import pandas as pd
import os
import csv

# 进度条展示
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# 写日志，为了后面绘制曲线图
from torch.utils.tensorboard import SummaryWriter

# 一些工具函数

固定的，只是不同的人写法不同。不需要修改

In [ ]:
def same_seed(seed): 
    '''修复随机数生成器种子以提高可重复性。'''
    # 将cudnn.deterministic置为True的话，每次返回的卷积算法将是确定的，即默认算法。
    torch.backends.cudnn.deterministic = True
    # 将cudnn.benchmark置为True将会让程序在开始时花费一点额外时间，为整个网络的每个卷积层搜索最适合它的卷积实现算法，进而实现网络的加速。
    torch.backends.cudnn.benchmark = False
    # 为numpy设置随机种子
    np.random.seed(seed)
    # 为CPU设置随机种子
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        # 为所有的GPU设置随机种子
        torch.cuda.manual_seed_all(seed)

def train_valid_split(data_set, valid_ratio, seed):
    '''将提供的训练数据拆分为训练集和验证集'''
    # 参数的意思： data_set总的数据集，valid_ratio验证集的比例，seed随机种子
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    # 随机划分 训练集 和 验证集
    # random_split随机将一个数据集分割成给定长度的 不重叠 的新数据集。可选择固定生成器以获得可复现的结果
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

def predict(test_loader, model, device):
    # 预测过程
    model.eval() # 将模型设置为评估模式。
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

# Dataset

In [ ]:
class COVID19Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    dataset
    '''
    def __init__(self, x, y=None):
        if y is None:
            self.y = y
        else:
            self.y = torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# 神经网络模型
通过修改下面的类来尝试不同的模型体系结构。

In [ ]:
class My_Model(nn.Module):
    # 创建模型
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1) # (B, 1) -> (B)
        return x

# 特征选择
通过修改以下功能来选择您认为有用的功能。

In [ ]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''选择有用的 特征 以执行回归'''
    y_train, y_valid = train_data[:,-1], valid_data[:,-1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:,:-1], valid_data[:,:-1], test_data

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [0,1,2,3,4] # TODO: Select suitable feature columns.
        
    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid

# 循环训练

In [ ]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean') # Define your loss function, do not modify this.

    # 定义优化算法 
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9) 

    writer = SummaryWriter() # Writer of tensoboard.
    # 如果不存在模型文件夹，就创建一个
    if not os.path.isdir('./models'):
        os.mkdir('./models') 

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)
            # 计算损失             
            loss = criterion(pred, y)
            # 反向传播
            loss.backward()
            # 迭代                     
            optimizer.step()                    
            step += 1
            # detach()分离tensor但是没有更改这个tensor时，并不会影响backward()
            loss_record.append(loss.detach().item())
            
            # 在 tqdm 进度条上显示当前epoch和loss。
            # set_description()设定的是前缀，set_postfix()设定的是后缀
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        # 切换为评估模式
        model.eval() 
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            # 不进行反向传播更新参数
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

# 配置
`config` 包含用于训练的超参数和保存模型的路径。

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # 随机种子（随便设置）
    'select_all': True,   # 是否使用所有功能。
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 100,     # epoch的次数          
    'batch_size': 256, 
    'learning_rate': 1e-5,  # 学习率            
    'early_stop': 400,    # 如果模型在这么多个连续的epoch内没有改进，请停止训练。     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

# Dataloader
从文件中读取数据并设置训练、验证和测试集。您不需要修改此部件。

In [ ]:
# Set seed for reproducibility
same_seed(config['seed'])


# train_data size: 2699 x 118 (id + 37 states + 16 features x 5 days) 
# test_data size: 1078 x 117 (without last day's positive rate)
train_data, test_data = pd.read_csv('./covid.train.csv').values, pd.read_csv('./covid.test.csv').values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

train_dataset, valid_dataset, test_dataset = COVID19Dataset(x_train, y_train), \
                                            COVID19Dataset(x_valid, y_valid), \
                                            COVID19Dataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

# 开始训练！

# 用 `tensorboard` 绘制曲线图 (可选)

`tensorboard` 是一种工具，可让您可视化训练进度。

如果此块未显示您的学习曲线，请等待几分钟，然后重新运行此块。加载日志记录信息可能需要一些时间。 

In [ ]:
model = My_Model(input_dim=x_train.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

In [ ]:
# 绘制曲线图
%reload_ext tensorboard
%tensorboard --logdir=./runs/

# 测试
The predictions of your model on testing set will be stored at `pred.csv`.

In [ ]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    with open(file, 'w') as fp:
        writer = csv.writer(fp)
        writer.writerow(['id', 'tested_positive'])
        for i, p in enumerate(preds):
            writer.writerow([i, p])

model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device) 
save_pred(preds, 'pred.csv')         

# Reference
This notebook uses code written by Heng-Jui Chang @ NTUEE (https://github.com/ga642381/ML2021-Spring/blob/main/HW01/HW01.ipynb)